In [ ]:
import boto3
import time
import psycopg2

In [ ]:
# Find the most visited URL 
# per country 
# per day 
# during a week of your choice (e.g. 2018-04-01 until 2018-04-08).

In [ ]:
# Connect to the AWS resources with your credentials
AWS_ACCESS_KEY = '<your-access-key>'
AWS_SECRET_KEY = '<your-secret-key>'

# define the database name created in Glue
ATHENA_DATABASE_NAME = 'christiana-athena-parquet'

# Define the output location
S3_OUTPUT_LOCATION = 's3://athena-learners-etl-bite05/christiana/'

# Connect to the Athena client
athena_client = boto3.client(
    'athena',
    region_name='eu-west-2',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY)

# Write the SQL query
sql_query = """
WITH visited_urls_count AS (
    SELECT
        server_request_country_code,
        event_url,
        COUNT(*) AS visit_count,
        DATE(datetime) AS visit_day
    FROM
        vod_clickstream 
    WHERE
        datetime >= CAST('2017-03-29' AS timestamp) 
        AND datetime < CAST('2017-04-05' AS timestamp)
    GROUP BY
        server_request_country_code,
        event_url,
        DATE(datetime)
),
ranked_urls as (SELECT
    server_request_country_code,
    event_url AS most_visited_url,
    visit_day,
    visit_count,
    ROW_NUMBER() OVER (PARTITION BY server_request_country_code, visit_day ORDER BY visit_count DESC) AS rank FROM visited_urls_count)
SELECT visit_day, server_request_country_code, most_visited_url, visit_count FROM ranked_urls
WHERE rank = 1
"""

# Execute the Athena query
query_execution = athena_client.start_query_execution(
    QueryString=sql_query,
    QueryExecutionContext={
        "Database": ATHENA_DATABASE_NAME
    },
    ResultConfiguration={
        "OutputLocation": S3_OUTPUT_LOCATION
    }
)

# Poll the query status until it is either successful or failed
query_status = "QUEUED"
query_execution_id = query_execution["QueryExecutionId"]

while query_status in ["QUEUED", "RUNNING"]:
    query_execution = athena_client.get_query_execution(
        QueryExecutionId=query_execution_id
    )
    query_status = query_execution["QueryExecution"]["Status"]["State"]
    if query_status == "FAILED":
        raise Exception("Athena query failed!")
    time.sleep(1)

# Retrieve the query results
results = athena_client.get_query_results(
    QueryExecutionId=query_execution_id
)["ResultSet"]["Rows"]

results

In [ ]:
conn = psycopg2.connect(database="etl_bites", user="", password="", host="localhost", port="5432")
cursor = conn.cursor()

# Create a new table to put the data that has been transformed
cursor.execute("""
    CREATE TABLE IF NOT EXISTS most_visited_urls (
        visit_day DATE,
        country_code VARCHAR(2),
        most_visited_url VARCHAR, 
        visit_count INTEGER,
        PRIMARY KEY (country_code, visit_day)
    );
""")

for row in results[1:]:
    # Check for valid data in each row
    try:
        visit_day = row["Data"][0].get("VarCharValue", None)
        country_code = row["Data"][1].get("VarCharValue", None)
        most_visited_url = row["Data"][2].get("VarCharValue", None)
        visit_count = row["Data"][3].get("VarCharValue", None)

        # Ensure none of the values are missing
        if not visit_day or not country_code or not most_visited_url or not visit_count:
            print(f"Skipping row due to missing data: {row}")
            continue
        
        visit_count = int(visit_count)  # Convert visit_count to an integer

        # Insert data into the database
        insert_query = """
            INSERT INTO most_visited_urls (visit_day, country_code, most_visited_url, visit_count)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (visit_day, country_code)
            DO UPDATE SET visit_count = EXCLUDED.visit_count;
        """
        cursor.execute(insert_query, (visit_day, country_code, most_visited_url, visit_count))

    except KeyError as e:
        print(f"KeyError: {e} in row: {row}")
    except Exception as e:
        print(f"Error inserting row: {row}, Error: {e}")
        conn.rollback()  # Rollback the transaction if there's an error

# Commit the changes and close the cursor and connection outside the loop
conn.commit()
cursor.close()
conn.close()
